In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.stats import binom
from scipy.stats import gamma
from matplotlib import pyplot as plt
from typing import Tuple, List, Callable, Any
from file_util import *

In [ ]:
# General settings
num_runs = 1
starting_seed = 0
seed_multiplier = 100

# Validator settings
num_nodes = 32
num_consensus = 2000
base_time_limit = 10000
node_processing_distribution = "exp"
node_processing_parameters = [3]
consensus_protocol = "HS"

## Fault settings
num_faults = 1
fault_type = "UR"
fault_parameters = []

# Network settings
## Switch settings
switch_processing_distribution = "degen"
switch_processing_parameters = [0]
message_channel_success_rate = 1

network_type = "Clique"
network_parameters = []

In [ ]:
# More utility methods for analysis
def get_minima(series: pd.Series):
    return series[(series < series.shift(1)) & (series < series.shift(-1))].iloc[0]

def get_minima_index(series: pd.Series):
    return series[(series < series.shift(1)) & (series < series.shift(-1))].index[0]

In [ ]:
def get_num_faults_data(num_faults: int, name: str, dir:str=DEFAULT_RESULTS_DIRECTORY) -> pd.Series:
    return get_fn_data(lambda name: name.get_btl(), lambda json: json[TOTAL_TIME_KEY], name, dir=dir,
        num_nodes=num_nodes, topology=network_type, protocol=consensus_protocol, 
        num_faults=num_faults, node_dist=node_processing_distribution) 

get_num_faults_data(0, "0_fault")

In [ ]:
lst_num_faults = [0, 1, 2, 3]
df = pd.DataFrame({str(x) + "_fault": get_num_faults_data(x, str(x) + "_fault") for x in lst_num_faults})
df = df.interpolate(method="linear")
df.plot(grid=True, style=".-", xlabel="base_time_limit", ylabel="time_to_consensus", title=consensus_protocol + " simulation")
plt.show()

In [ ]:
# gradients
# 0: 0, 1: 0.05, 2: 0.125, 3: 0.225
# gradients = {0: 0, 1: 0.0625, 2: 0.142, 3: 0.245, 4: 0.383}
gradients = {0: 0, 1: 0.0714, 2: 0.167, 3: 0.3, 4: 0.5}
names = [str(x) + "_fault" for x in lst_num_faults]
minima = [get_minima(df[name]) for name in names]
minima_index = [get_minima_index(df[name]) for name in names]
minima, minima_index

def get_ibft_right_line(num_faults: int):
    index = lst_num_faults.index(num_faults)
    return lambda t : minima[index] + (t - minima_index[index]) * gradients[num_faults]

df["test"] = df.index.map(get_ibft_right_line(3))
df.plot(style=".-", grid=True)


In [ ]:
### IBFT WORK
def get_round_change_probs(num_faults: int, name: str):
    return get_fn_data(num_nodes, network_type, "ibft", num_faults, node_processing_distribution, 
                       lambda json: min(json[FASTEST_MESSAGE_MAP][PREPREPARED] - 1, 1), name)

###
def time5(t: float, n: int, f: int, mu: float) -> float:
    p = get_success_probability(t, n, f, mu) 
    r = f / n
    t_fault_first = (r + 2 * r * (r - (1 / n)) + 4 * r * (r - 1 / n) * (r - 2 / n)) * t 

    max_f = (n - 1) // 3

    t_working_fail = (n - f) / n * (1 - p) * (t + t_fault_first * 2 + ((max_f - f + 1) / mu) + (3 * n - 2 * f - max_f) / mu + (n - f) / mu * (1 - r)) # 3.1 is first round new round penalty, additional n - f is penalty approx of changing halfway through
    t_fault_first += r * ((2 * n - max_f - f) / mu) 
    t_working_success = (n - f) / n * p * min(t, ibft_te2e(n, mu) - f / mu)
    return t_fault_first + t_working_fail + t_working_success

def ibft_te2e(n: int, mu: float):
    return (10 * n + 2) * (2 * n + 1) / (mu * (7 + 11 * n - np.sqrt(37 + 70 * n + n * 2)))

def get_success_mean(n: int, f: int, mu: float) -> float:
    max_f = (n - 1) // 3
    return ((max_f - f) / 2 + 2 * n - max_f - f) / mu
    # return (2 * n - max_f - f) / mu

def get_success_stdev(n: int, f: int, mu: float, t: float) -> float:
    max_f = (n - 1) // 3
    return np.sqrt((1 / (mu ** 2)) * (2 * n - max_f - f + (max_f - f) / 2) * (2 + 1 / (n - f)))
    # return np.sqrt((1 / (mu ** 2)) * (2 * n - max_f - f) * (2 + 1 / (n - f)))

def get_success_probability(t: float, n: int, f: int, mu: float) -> float:
    return norm.cdf(t, loc=get_success_mean(n, f, mu), scale=get_success_stdev(n, f, mu, t))

def ibft_reco(n: int, f: int, mu: float, sigma: float):
    max_f = (n - 1) // 3
    m = ((max_f - f) / 2 + 2 * n - max_f - f)
    return m * mu + sigma * np.sqrt(m * (2 + 1 / (n - f))) * 3 

def ibft_test(f: int):
    round_change_probs = get_round_change_probs(f, "round_change_prob_" + str(f)) 
    consensus_times = get_num_faults_data(f, "faults_" + str(f))

    df = pd.DataFrame({"RC_PROB": round_change_probs, TOTAL_TIME_KEY: consensus_times})
    df["new_prob"] = df.index.map(lambda t: (num_nodes - f) / num_nodes - (num_nodes - f) / num_nodes * norm.cdf(t, loc=get_success_mean(num_nodes, f, 3), scale=get_success_stdev(num_nodes, f, 3, t)))
    # df[["RC_PROB", "new_prob"]].plot(style=".-", grid=True, xlabel="base time limit", ylabel="probability of round change")
    df["prediction"] = df.index.map(lambda t: time5(t, num_nodes, f, 3))
    df[[TOTAL_TIME_KEY, "prediction"]].plot(grid=True, style=".-", title="Consensus time vs prediction with " + str(f) + " faults", figsize=(10, 5))
    plt.axvline(ibft_reco(num_nodes, f, 1/3, 1/3))
    plt.show()

# for i in [0, 1, 2, 3, 4]:
for i in [0, 2, 4, 8]:
    max_f=i
    ibft_test(i)
# df.new_prob.plot(style=".-", grid=True)

In [ ]:
def hs_time(n, t, mu, te2e):
    max_f = (n - 1) // 3
    p = gamma.cdf(t, 4 * n + 3 - max_f, scale=(1/mu))
    # print(p)
    t_fail = (1 - p) * t 
    # p2 = gamma.cdf(2 * t, 4 * n + 3  - max_f, scale=(1/mu))
    # t_fail += (1 - p) * (1 - p2) * (2 * t)
    t_succeed = 1 * te2e
    # p3 = gamma.cdf(4 * t, 4 * n + 3 - f, scale=(1/mu))
    # t_fail += (1 - p) * (1 - p2) * (1 - p3) * (4 * t)
    t_total = t_fail + t_succeed
    return t_total

def hs_time_fault(n, t, rate, f):
    max_f = (n - 1) // 3
    m = (4 * n + 4 - max_f - 3 * f) 
    te2e = m / rate 

    p_fault = f / n
    t_penalty_first_fault = 0 
    for i in range(1, f + 1):
        t_penalty_first_fault += p_fault ** i * t * (2 ** (i - 1)) 

    p = gamma.cdf(t, m - 1, scale=(1/rate))
    t_fail = (1 - p) * (t + t_penalty_first_fault * 2)
    t_total = t_fail * (n - f) / n + te2e 

    # print(t_total, t_fail, t_succeed, t_penalty_first_fault)
    return t_total + t_penalty_first_fault

# Testing recommendation
def hs_reco(n, f, mu, sigma):
    max_f = (n - 1) // 3
    m = 4 * n - max_f - 3 * f
    sigma_m = np.sqrt(m) * sigma 
    return (m * mu + 3 * sigma_m) 

def range_max_estimate(n, mu):
    max_f = (n - 1) // 3
    m = 4 * n - max_f 
    return int(m * mu * 1.75)

def range_min_estimate(n, mu):
    # return int(range_max_estimate(n, mu) / 1.75 / 2)
    return 0

def get_hs_minima(n, mu, frac):
    max_f = (n - 1) // 3
    f = int(frac * max_f)
    top = range_max_estimate(n, mu)
    bot = range_min_estimate(n, mu)
    if frac == 0:
        return hs_time_fault(n, top, 1/mu, f) 
    rg = [bot + (top - bot) / 1000 * i for i in range(1000)] 
    lst = [hs_time_fault(n, t, 1/mu, max_f * frac // 2) for t in rg] 
    s = pd.Series(lst, index=rg)
    temp = s[(s < s.shift(1)) & (s < s.shift(-1))]
    return temp.iloc[0]

def get_num_faults_from_frac(n, frac):
    return int((n - 1) // 3 * frac / 2)

def hs_estimate_left_line(n, f, mu, t):
    max_f = (n - 1) // 3
    m = 4 * n - max_f - 3 * f
    te2e = m * mu
    # gradient = n / (n - 2 * f) 
    # return te2e + gradient * t
    one = te2e + sum((f / n) ** (i - 1) * t * 2 ** (i - 1) for i in range(1, f + 1))
    one *= f / n
    two = te2e + sum((f / n) ** i * (t * 2 ** i) for i in range(f + 1))
    two *= (n - f) / n
    return one + two


def hs_estimate_right_line(n, f, mu, t):
    max_f = (n - 1) // 3
    m = 4 * n - max_f - 3 * f
    mean = m * mu
    gradient = f / (n - 2 * f)
    return mean + gradient * t

def full_estimate(n, f, mu, sigma, t):
    max_f = (n - 1) // 3
    m = 4 * n - max_f - 3 * f
    mean = m * mu
    sigma_m = np.sqrt(m) * sigma
    bot_boundary = mean - 2 * sigma_m
    top_boundary = mean + 2 * sigma_m
    y1 = hs_estimate_left_line(n, f, mu, bot_boundary)
    y2 = hs_estimate_right_line(n, f, mu, top_boundary)
    if t > top_boundary:
        return hs_estimate_right_line(n, f, mu, t)
    elif t < bot_boundary:
        return hs_estimate_left_line(n, f, mu, t)
    else:
        gradient = (y2 - y1) / (top_boundary - bot_boundary)
        intercept = y2 - gradient * top_boundary
        return gradient * t + intercept


In [ ]:
index = [8, 12, 16, 32, 48, 64, 128, 256, 512, 1024]
max_fault = pd.Series(index, index=index).apply(lambda n: get_hs_minima(n, 1/3, 1))
no_fault = pd.Series(index, index=index).apply(lambda n: get_hs_minima(n, 1/3, 0))
pd.DataFrame({"Max faults": max_fault, "No faults": no_fault}).plot(style=".-", title="HotStuff: Max number of faults - time to consensus with optimal BTL", ylabel="Time to consensus", xlabel="Number of validators")

In [ ]:
print(num_nodes, num_faults)
df["prediction"] = df.index.map(lambda t: hs_time_fault(32, t, node_processing_parameters[0], 2))
# df["prediction"] = df.index.map(lambda t: full_estimate(num_nodes, num_faults, 1/3, 1/3, t))
df.plot(style=".-", figsize=(10, 5), grid=True, xlabel="base_time_limit", ylabel="time per consensus")

In [ ]:
test_num_nodes = [16, 32, 64, 128, 256, 512, 1024, 2048]
test_mu = 1 / 3
test_num_faults = [2]
plot_num_nodes = 2048
rg = range(range_min_estimate(plot_num_nodes, test_mu), range_max_estimate(plot_num_nodes, test_mu))
dic = {str(plot_num_nodes) + "_" + str(j): [hs_time_fault(plot_num_nodes, t, 1/test_mu, get_num_faults_from_frac(plot_num_nodes, j)) 
                               for t in range(range_min_estimate(plot_num_nodes, test_mu), range_max_estimate(plot_num_nodes, test_mu))]
                               for j in test_num_faults}
dic["test"] = [full_estimate(plot_num_nodes, get_num_faults_from_frac(plot_num_nodes, 2), test_mu, test_mu, t) for t in rg]
pd.DataFrame(dic, index=rg).plot(figsize=(15, 10), grid=True, ylabel="time to consensus", xlabel="base time limit")
[plt.axvline(hs_reco(plot_num_nodes, get_num_faults_from_frac(plot_num_nodes, f), test_mu, test_mu)) for f in test_num_faults]


In [ ]:
t = 20
ibft_fn = lambda f: time5(t, 16, f, 3)
hs_fn = lambda f: hs_time_fault(16, t, 3, f)

index = pd.Series(range(6))
ibft_pred = index.map(ibft_fn)
hs_pred = index.map(hs_fn)
pd.DataFrame({"ibft": ibft_pred, "hs": hs_pred}).plot(style=".-")

In [ ]:
index = range(20,70)
n = 32
max_f = (n - 1) // 3
mu_hs = 3
mu_ibft = mu_hs * ((2 * n + 1) / (4 * n - max_f))

f = 9

ibft_pred = pd.Series(map(lambda btl: time5(btl, n, f, mu_ibft), index), index=index, name="ibft_pred")
hs_pred = pd.Series(map(lambda btl: hs_time_fault(n, btl, mu_hs, f), index), index=index, name="hs_pred") 

pd.DataFrame([ibft_pred, hs_pred]).transpose().plot(style=".-")